# Tugas 1 : Crawling Berita

**Apa itu crawling ?**


Perayapan data, atau data crawling adalah sebuah proses yang menggali jauh ke dalam world wide web atau target tertentu untuk mengambil data. Perayapan data juga didefinisikan sebagai proses otomatis untuk mengumpulkan dan mengindeks data dari berbagai sumber.

Aktivitas ini dilakukan oleh bot atau software yang disebut dengan crawler. Data yang diambil dari hasil crawling ini pada umumnya akan dianalisis, dijadikan bahan pengembangan sistem, atau bahkan dijadikan data penelitian tertentu.

Proses pengambilan data dari proses perayapan ini diawali dengan crawler yang mulai memasuki target yang telah ditentukan. Setelah itu, crawler akan mengambil data-data penting yang dibutuhkan.

Pada umumnya, data-data yang diambil dari proses ini berupa spesifikasi produk, harga, kategori, dan lain-lain.

**Fungsi Data Crawling**

1. Data Statistik

  Fungsi data crawling yang pertama adalah untuk kebutuhan data statistik. Pada dasarnya, proses perayapan data memang berfungsi untuk mengumpulkan data-data tertentu, termasuk informasi statistik.

  Data statistik yang diperoleh ini nantinya akan digunakan sebagai bahan analisis. Data statistik yang pada umumnya diambil melalui proses crawling adalah data analisis pasar, analisis calon konsumen, dan lain-lain.

2. Membandingkan Detail Sebuah Produk

  Ketika Anda mencari sebuah produk di internet, kemudian berniat untuk membandingkan detail produk tersebut seperti spesifikasi, harga, serta kategorinya, di sinilah perayapan data berperan.

  Dengan bantuan perayapan data, Anda bisa menemukan produk di mesin pencari, seiring dengan preferensi lainnya yang terkait dengan produk tersebut.

**Source**

Pada proses crawling ini sumber data diambil dari  https://health.detik.com, Detik Health adalah salah satu sub-kanal dari situs berita Detik.com yang khusus membahas topik kesehatan. Detik Health bertujuan untuk menyediakan informasi yang bermanfaat bagi pembaca yang ingin menjaga kesehatan atau mencari solusi atas masalah kesehatan mereka. Data yang diambil diantaranya :


*   Judul

* Isi berita

* Tanggal

* Kategori Berita

pada tugas crawling berita kali ini hanya mengambil data sampel berita sebanyak 10 artikel


**Proses Crawling**

Dalam proses crawling ini saya menggunakan beberapa library dan diantara library yang penting di import adalah *BeautifulSoup* yang berfungsi sebagai library crawler

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


Membuat fungsi filter konten
Berfungsi untuk Mem-filter dari elemen-elemen HTML pada berita yang tidak diinginkan , Contoh kasus seperti iklan, daftar isi, gambar , link sisipan, dll

In [6]:

# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"


Membuat fungsi untuk melakukan crawling data pada situs web Detik.com.
Fungsi ini mengambil data berupa judul berita, tanggal publikasi, isi berita dan kategori berita yang terdapat di halaman tersebut.


In [7]:
# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return  # Menghentikan proses jika jumlah artikel sudah mencapai minimum yang diinginkan

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        # Bersihkan konten menggunakan fungsi clean_content
        content = clean_content(content_element)

        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        kategori_list.append(kategori)

        if len(judul) <= 10:
            print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://health.detik.com/berita-detikhealth/indeks",
             "https://sport.detik.com/indeks",
]
categories = ["Kesehatan",
              "Olahraga",
]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 50

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}/{page}"
        get_data(url, category, min_articles_per_category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman
        page += 1

# Membuat dataframe dari list data
df = pd.DataFrame({"judul": judul, "tanggal": tanggal, "isi": isi, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df.to_csv("data_berita_detik.csv", index=False)



Jangan Sembarangan! Waspadai Pemicu Cedera saat Lari yang Sering Dialami Pemula
Keseringan Pakai TWS? Waspada Gangguan Pendengaran NIHL, Ini Tandanya
Jogging Pakai TWS Aman Nggak Sih? Ini Plus-Minusnya Menurut Dokter
5 Kebiasaan Simpel yang Bisa Bikin Panjang Umur, Salah Satunya Menikmati Makanan
Rajin Sit-up Saja Nggak Bikin Lemak di Perut Hilang, Ini yang Kerap Terlupakan
Waduh! Peneliti di China Temukan Virus Baru Akibat Kutu, Efeknya Bisa ke Otak
Banjir Keringat saat Olahraga Tanda Tubuh Bakar Lemak? Begini Faktanya
Kemendikbudristek Turun Tangan Urus Bullying PPDS Undip, Bentuk Tim Pencari Fakta
Terbukti Lewat Studi, Doyan Traveling Ternyata Bisa Bikin Panjang Umur!
9 Kelompok Orang yang Tidak Disarankan Minum Kopi


In [8]:
df=pd.read_csv("data_berita_detik.csv")
df.head(10)

,judul,tanggal,isi,kategori
0,Jangan Sembarangan! Waspadai Pemicu Cedera saa...,"Sabtu, 07 Sep 2024 19:00 WIB",Jakarta - Olahraga lari dalam beberapa waktu t...,Kesehatan
1,Keseringan Pakai TWS? Waspada Gangguan Pendeng...,"Sabtu, 07 Sep 2024 18:00 WIB",Jakarta - Adanya teknologi earphone nirkabel a...,Kesehatan
2,Jogging Pakai TWS Aman Nggak Sih? Ini Plus-Min...,"Sabtu, 07 Sep 2024 17:00 WIB",Jakarta - Tren penggunaan earphone nirkabel at...,Kesehatan
3,5 Kebiasaan Simpel yang Bisa Bikin Panjang Umu...,"Sabtu, 07 Sep 2024 14:00 WIB",Jakarta - Hidup sehat dengan umur yang panjang...,Kesehatan
4,Rajin Sit-up Saja Nggak Bikin Lemak di Perut H...,"Sabtu, 07 Sep 2024 13:00 WIB",Jakarta - Spesialis kedokteran olahraga dr And...,Kesehatan
5,Waduh! Peneliti di China Temukan Virus Baru Ak...,"Sabtu, 07 Sep 2024 12:00 WIB",Jakarta - Ilmuwan menemukan virus baru di Chin...,Kesehatan
6,Banjir Keringat saat Olahraga Tanda Tubuh Baka...,"Sabtu, 07 Sep 2024 11:00 WIB",Jakarta - Banyaknya keringat yang keluar masih...,Kesehatan
7,Kemendikbudristek Turun Tangan Urus Bullying P...,"Sabtu, 07 Sep 2024 09:45 WIB","Jakarta - Kementerian Pendidikan, Kebudayaan, ...",Kesehatan
8,"Terbukti Lewat Studi, Doyan Traveling Ternyata...","Sabtu, 07 Sep 2024 09:01 WIB",Jakarta - Peneliti dari Edith Cowan University...,Kesehatan
9,9 Kelompok Orang yang Tidak Disarankan Minum Kopi,"Sabtu, 07 Sep 2024 07:00 WIB",Jakarta - Kopi menjadi pilihan minuman untuk m...,Kesehatan
